In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys

In [2]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
PAGE_NUM = []
def find_page_num():

    html = getAndParseURL("https://www.zingat.com/karsiyaka-satilik-daire")
    for i in range(1,3):
        PAGE_NUM.append("https://www.zingat.com/karsiyaka-satilik-daire" + "?page=" + str(i))
    return PAGE_NUM
find_page_num()

['https://www.zingat.com/karsiyaka-satilik-daire?page=1',
 'https://www.zingat.com/karsiyaka-satilik-daire?page=2']

In [4]:
ALL_PRODUCT = []
def all_product():
    for i in PAGE_NUM[::]:
        html = getAndParseURL(i)
        for result in html.findAll("a",{"class":"zl-card-inner"}):
            ALL_PRODUCT.append("https://www.zingat.com" + result.get("href"))
    return ALL_PRODUCT
all_product()

['https://www.zingat.com/karsiyaka-goncalar-mah-girne-bulvari-yakini-satilik-3-1-daire-4926639i',
 'https://www.zingat.com/re-max-targettan-karsiyaka-aksoyda-sifir-daire-4986558i',
 'https://www.zingat.com/bornova-turyaptan-karsiyaka-bahceli-evlerde-3-1-satilik-daire-4810424i',
 'https://www.zingat.com/remax-target-ayse-ayhandan-demirkopru-de-satilik-4-2-dubleks-4857535i',
 'https://www.zingat.com/karsiyaka-bostanlida-satilik-ebeveyn-banyolu-3-1-daire-4955410i',
 'https://www.zingat.com/karsiyaka-bahriye-ucok-mah-ara-kat-satilik-3-1-daire-4984637i',
 'https://www.zingat.com/karsiyaka-aksoyda-asansorlu-genis-ve-ferah-satilik-3-1-daire-4984642i',
 'https://www.zingat.com/semikler-pristina-arkasi-satilik-2-1-daire-4984613i',
 'https://www.zingat.com/ornekkoyde-kapali-mutfak-satilik-3-1-daire-4935335i',
 'https://www.zingat.com/bostanlida-muhtesem-konumlu-3-1-satilik-daire-4981574i',
 'https://www.zingat.com/karsiyaka-yali-mahallesinde-2-1-kelepir-satilik-dubleks-daire-4981720i',
 'https:/

In [5]:
RESULT = []
for rslt in ALL_PRODUCT[:]:

    html = getAndParseURL(rslt)
    house_description = html.find("div", {"class":"col-xs-12"}).h1.text
    neighborhood = html.find("div", {"class":"detail-location-path"}).h2.text.replace(",","").strip()
    price = html.find("div", {"class":"col-md-3 price-info-text"}).find("strong",{"itemprop":"price"}).text.replace("TL","").strip()
    net_m2 = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[2].find("span").text.replace("m²","").strip()
    gross_m2 = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[3].find("span").text.replace("m²","").strip()
    room_livingroom = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[4].find("span").text.strip()
    room = html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[4].find("span").text.strip()[0]
    kredi_durumu= html.find("ul", {"class":"row attribute-detail-list"}).findAll("li",{"class":"col-md-6"})[-1].find("span").text.strip()

    RESULT.append([house_description,neighborhood,price,net_m2,gross_m2,room_livingroom,room,kredi_durumu])
    
columns = ["İLAN_ADI","MAHALLE", "FİYAT","NET(m²)","BRÜT(m²)","ODA_SALON_SAYISI","ODA_SAYISI","KREDI_UYGUNLUGU"]
df = pd.DataFrame.from_records(RESULT, columns=columns)

In [6]:
df

,İLAN_ADI,MAHALLE,FİYAT,NET(m²),BRÜT(m²),ODA_SALON_SAYISI,ODA_SAYISI,KREDI_UYGUNLUGU
0,Karşıyaka Goncalar Mah Girne Bulvarı Yakını Sa...,Goncalar Karşıyaka\n İzmir,3.750.000,97,110,3+1,3,Evet
1,RE/MAX Target'tan Karşıyaka Aksoyda Sıfır Daire,Aksoy Karşıyaka\n İzmir,8.635.000,80,100,3+1,3,Hayır
2,BORNOVA TURYAP'TAN KARŞIYAKA BAHÇELİ EVLERDE 3...,Bahçelievler Karşıyaka\n İzmir,4.650.000,140,160,3+1,3,Evet
3,Remax Target Ayşe Ayhandan Demirköprü de satıl...,Şemikler Karşıyaka\n İzmir,8.700.000,145,160,4+2,4,Evet
4,Karşıyaka Bostanlı'da Satılık Ebeveyn Banyolu ...,Bostanlı Karşıyaka\n İzmir,5.950.000,120,130,3+1,3,Evet
5,Karşıyaka Bahriye Üçok Mah. Ara Kat Satılık 3+...,Bahriye Üçok Karşıyaka\n İzmir,3.150.000,110,120,3+1,3,Evet
6,Karşıyaka AKSOY'da Asansörlü Geniş ve Ferah Sa...,Aksoy Karşıyaka\n İzmir,3.800.000,135,145,3+1,3,Evet
7,Şemikler Priştina Arkasi Satılık 2+1 daire,Şemikler Karşıyaka\n İzmir,2.480.000,65,75,2+1,2,Evet
8,ÖRNEKKÖY'DE KAPALI MUTFAK SATILIK 3+1 DAİRE,Örnekköy Karşıyaka\n İzmir,2.350.000,120,140,3+1,3,Evet
9,BOSTANLIDA MUHTEŞEM KONUMLU 3+1 SATILIK DAİRE,Atakent Karşıyaka\n İzmir,5.790.000,110,130,3+1,3,Evet


In [7]:
df.to_csv("zingat_house_price_prediction.csv", encoding="utf-8")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   İLAN_ADI          42 non-null     object
 1   MAHALLE           42 non-null     object
 2   FİYAT             42 non-null     object
 3   NET(m²)           42 non-null     object
 4   BRÜT(m²)          42 non-null     object
 5   ODA_SALON_SAYISI  42 non-null     object
 6   ODA_SAYISI        42 non-null     object
 7   KREDI_UYGUNLUGU   42 non-null     object
dtypes: object(8)
memory usage: 2.8+ KB


In [9]:
df.isnull().sum()

İLAN_ADI            0
MAHALLE             0
FİYAT               0
NET(m²)             0
BRÜT(m²)            0
ODA_SALON_SAYISI    0
ODA_SAYISI          0
KREDI_UYGUNLUGU     0
dtype: int64

In [10]:
df["MAHALLE"].unique()

array(['Goncalar Karşıyaka\n İzmir', 'Aksoy Karşıyaka\n İzmir',
       'Bahçelievler Karşıyaka\n İzmir', 'Şemikler Karşıyaka\n İzmir',
       'Bostanlı Karşıyaka\n İzmir', 'Bahriye Üçok Karşıyaka\n İzmir',
       'Örnekköy Karşıyaka\n İzmir', 'Atakent Karşıyaka\n İzmir',
       'Yalı Karşıyaka\n İzmir', 'İmbatlı Karşıyaka\n İzmir',
       'Mavişehir Karşıyaka\n İzmir', 'Demirköprü Karşıyaka\n İzmir',
       'Mustafa Kemal Karşıyaka\n İzmir', 'Dedebaşı Karşıyaka\n İzmir',
       'İnönü Karşıyaka\n İzmir', 'Tuna Karşıyaka\n İzmir'], dtype=object)